In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
from math import log2
%matplotlib inline

from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_poisson_deviance
from sklearn.metrics import mean_gamma_deviance
from sklearn.metrics import median_absolute_error
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.pipeline import make_pipeline
import time
import matplotlib as mpl

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor



from sklearn.preprocessing import StandardScaler,PowerTransformer,MinMaxScaler,QuantileTransformer,normalize
from sklearn.model_selection import train_test_split, learning_curve, ShuffleSplit
from sklearn.feature_selection import VarianceThreshold, SelectKBest
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from sklearn.model_selection import train_test_split

import pathlib
import os
import sys

# for selection the right path
import os,sys,inspect,pathlib
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

from common.model_trainer_reg import *
from common.regression_plotfunctions import *


from GD.LinearRegression import LinearRegression
from KNN.KNNRegressor import KNNRegressor

import metro_preprocessing


In [2]:
# Custom functions

module_path = pathlib.Path(os.getcwd()).parent
if str(module_path) not in sys.path:
    sys.path.append(str(module_path))
print(sys.path)

from common.dataset_grabber import get_data_path

['/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2', '/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2', '/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2', '/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2', '/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2/Metro', '', '/home/alexander/ngsuite/ngsolve-install/lib/python3/dist-packages', '/home/alexander/ngsuite/ngsolve-install/bin', '/home/alexander/.local/bin', '/usr/local/sbin', '/usr/local/bin', '/usr/sbin', '/usr/bin', '/sbin', '/bin', '/usr/games', '/usr/local/games', '/snap/bin', '/home/alexander/.vscode/extensions/ms-toolsai.jupyter-2020.11.399280825/pythonFiles', '/home/alexander/.vscode/extensions/ms-toolsai.jupyter-2020.11.399280825/pythonFiles/lib/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/home/alexander/.local/lib/python3.7/site-packages', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '

# Train and Test

In [3]:
datapath = get_data_path("Metro", "Metro_Interstate_Traffic_Volume.csv")
display(datapath)

PosixPath('/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2/Datasets/Metro/Metro_Interstate_Traffic_Volume.csv')

In [4]:
df_raw = pd.read_csv(datapath)
df_raw = df_raw.sample(10000)

In [5]:
X, Y = metro_preprocessing.preprocessing(df_raw, transform = True)

## SGD-Regression

In [6]:
MODEL = SGDRegressor
NAME = "SGD"
PATH = "out/"+NAME+"/"
params = {"alpha" : [0.0001]}
n_splits = 7
test_size = 0.3

modeltrainer = ModelTrainer(MODEL, params, X, Y, thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = test_size)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "sklearn_TTS_SGD.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = test_size, random_state = 42)
results = modeltrainer.retResults(PATH + "sklearn_CV_SGD.csv")
display(results)

,alpha,train_time,inference_time,R2_score,RMSE,D,N,n_test,n_train
0,0.0001,0.009896,0.000436,0.22421,0.795015,3,9998,3000,6998


,alpha,train_time,inference_time,k,R2_score,RMSE,D,N,n_test,n_train
0,0.0001,0.014345,0.000884,1,0.227217,0.791933,3,9998,3000,6998
1,0.0001,0.016699,0.000498,2,0.222746,0.797807,3,9998,3000,6998
2,0.0001,0.027081,0.001530,3,0.207370,0.810532,3,9998,3000,6998
3,0.0001,0.018535,0.002119,4,0.230082,0.795330,3,9998,3000,6998
4,0.0001,0.021560,0.001149,5,0.222082,0.784231,3,9998,3000,6998
5,0.0001,0.026385,0.001406,6,0.215902,0.806432,3,9998,3000,6998
6,0.0001,0.022080,0.000425,7,0.211171,0.818548,3,9998,3000,6998


## My SGD-Regression

In [7]:
MODEL = LinearRegression
params = {"alpha" : [0.0001], "max_iter": [1000]}


modeltrainer = ModelTrainer(MODEL, params, X, Y, thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = test_size)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "my_TTS_SGD.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = test_size, random_state = 42)
results = modeltrainer.retResults(PATH + "my_CV_SGD.csv")
display(results)

,alpha,max_iter,train_time,inference_time,R2_score,RMSE,D,N,n_test,n_train
0,0.0001,1000,0.266454,0.000104,0.228596,0.79052,3,9998,3000,6998


,alpha,max_iter,train_time,inference_time,k,R2_score,RMSE,D,N,n_test,n_train
0,0.0001,1000,0.257006,0.000114,1,0.228596,0.790520,3,9998,3000,6998
1,0.0001,1000,0.205121,0.000104,2,0.223216,0.797324,3,9998,3000,6998
2,0.0001,1000,0.204670,0.000115,3,0.209049,0.808815,3,9998,3000,6998
3,0.0001,1000,0.210325,0.000104,4,0.232336,0.793000,3,9998,3000,6998
4,0.0001,1000,0.200077,0.000123,5,0.224240,0.782056,3,9998,3000,6998
5,0.0001,1000,0.199375,0.000103,6,0.217912,0.804365,3,9998,3000,6998
6,0.0001,1000,0.194887,0.000113,7,0.218378,0.811069,3,9998,3000,6998


## KNN-Regression

In [8]:
MODEL = KNeighborsRegressor
params = {"weights" : ["uniform"]}
NAME = "KNN"
PATH = "out/"+NAME+"/"


modeltrainer = ModelTrainer(MODEL, params, X, Y, thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = test_size)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "sklearn_TTS_KNN.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = test_size, random_state = 42)
results = modeltrainer.retResults(PATH + "sklearn_CV_KNN.csv")
display(results)

,weights,train_time,inference_time,R2_score,RMSE,D,N,n_test,n_train
0,uniform,0.107617,0.298849,0.610908,0.398735,3,9998,3000,6998


,weights,train_time,inference_time,k,R2_score,RMSE,D,N,n_test,n_train
0,uniform,0.105067,0.293494,1,0.610908,0.398735,3,9998,3000,6998
1,uniform,0.091586,0.293724,2,0.608402,0.401953,3,9998,3000,6998
2,uniform,0.093280,0.298966,3,0.600789,0.408227,3,9998,3000,6998
3,uniform,0.098515,0.294636,4,0.622333,0.390132,3,9998,3000,6998
4,uniform,0.090529,0.296565,5,0.616649,0.386462,3,9998,3000,6998
5,uniform,0.089042,0.289341,6,0.585727,0.426073,3,9998,3000,6998
6,uniform,0.091970,0.297257,7,0.629310,0.384656,3,9998,3000,6998


## my KNN-Regression

In [9]:
MODEL = KNNRegressor
params = {"n_neighbors" : [5],
           "p": [2]}#,
           #'weights': ['uniform']}
NAME = "KNN"
PATH = "out/"+NAME+"/"


modeltrainer = ModelTrainer(MODEL, params, X, Y, thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = test_size)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "my_TTS_KNN.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = test_size, random_state = 42)
results = modeltrainer.retResults(PATH + "my_CV_KNN.csv")
display(results)

chunking...


,n_neighbors,p,train_time,inference_time,R2_score,RMSE,D,N,n_test,n_train
0,5,2,0.000011,2.369871,0.610908,0.398735,3,9998,3000,6998


chunking...
chunking...
chunking...
chunking...
chunking...
chunking...
chunking...


,n_neighbors,p,train_time,inference_time,k,R2_score,RMSE,D,N,n_test,n_train
0,5,2,0.000007,2.385087,1,0.610908,0.398735,3,9998,3000,6998
1,5,2,0.000007,2.380238,2,0.608402,0.401953,3,9998,3000,6998
2,5,2,0.000007,2.369374,3,0.600789,0.408227,3,9998,3000,6998
3,5,2,0.000006,2.376965,4,0.622333,0.390132,3,9998,3000,6998
4,5,2,0.000007,2.410426,5,0.616678,0.386433,3,9998,3000,6998
5,5,2,0.000010,2.369179,6,0.585727,0.426073,3,9998,3000,6998
6,5,2,0.000006,2.364995,7,0.629310,0.384656,3,9998,3000,6998


## RF-Regression

In [10]:
MODEL = RandomForestRegressor
params = {"n_estimators" : [100]}
NAME = "RF"
PATH = "out/"+NAME+"/"



modeltrainer = ModelTrainer(MODEL, params, X, Y, thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = test_size)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "sklearn_TTS_RF.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = test_size, random_state = 42)
results = modeltrainer.retResults(PATH + "sklearn_CV_RF.csv")
display(results)

,n_estimators,train_time,inference_time,R2_score,RMSE,D,N,n_test,n_train
0,100,3.22781,0.079263,0.811926,0.192735,3,9998,3000,6998


,n_estimators,train_time,inference_time,k,R2_score,RMSE,D,N,n_test,n_train
0,100,3.226903,0.080765,1,0.810332,0.194368,3,9998,3000,6998
1,100,3.160897,0.080057,2,0.811498,0.193487,3,9998,3000,6998
2,100,3.265276,0.102579,3,0.785349,0.219500,3,9998,3000,6998
3,100,3.505711,0.090657,4,0.825893,0.179853,3,9998,3000,6998
4,100,3.316311,0.096711,5,0.813946,0.187564,3,9998,3000,6998
5,100,3.256614,0.082972,6,0.806605,0.198903,3,9998,3000,6998
6,100,3.371263,0.090773,7,0.833145,0.173141,3,9998,3000,6998


## DT-Regression

In [11]:
MODEL = DecisionTreeRegressor
params = {"criterion": ["mse"]}
NAME = "DT"
PATH = "out/"+NAME+"/"



modeltrainer = ModelTrainer(MODEL, params, X, Y, thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = test_size)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "sklearn_TTS_DT.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = test_size, random_state = 42)
results = modeltrainer.retResults(PATH + "sklearn_CV_DT.csv")
display(results)

,criterion,train_time,inference_time,R2_score,RMSE,D,N,n_test,n_train
0,mse,0.067321,0.001415,0.665031,0.34327,3,9998,3000,6998


,criterion,train_time,inference_time,k,R2_score,RMSE,D,N,n_test,n_train
0,mse,0.061836,0.001225,1,0.672858,0.335249,3,9998,3000,6998
1,mse,0.057998,0.001457,2,0.665373,0.343475,3,9998,3000,6998
2,mse,0.058254,0.001375,3,0.640348,0.367775,3,9998,3000,6998
3,mse,0.051903,0.001043,4,0.684668,0.325740,3,9998,3000,6998
4,mse,0.050547,0.001130,5,0.647728,0.355131,3,9998,3000,6998
5,mse,0.049521,0.001175,6,0.676761,0.332446,3,9998,3000,6998
6,mse,0.049736,0.001262,7,0.682996,0.328947,3,9998,3000,6998
